# Przeprowadzenie eksperymentu AB

Zgodnie ze sprawozdaniem eksperyment AB przyjmuje następującą formę: _"Co miesiąc system będzie przydzielał użytkowników do dwóch różnych podgrup (grupy kontrolnej i grupy doświadczalnej). W rezultacie otrzymamy dwie grupy w każdym miesiącu: grupę kontrolną, korzystającą z portalu w sposób niezmieniony oraz grupę doświadczalną, która korzysta z portalu, ale z zastosowaniem naszego modelu. Pod koniec miesiąca zbierane są dane dotyczące czasu spędzanego przez użytkowników w obu grupach."_


### Podział użytkowników na grupe kontrolną i doświadczalną

Przy podziale zależy nam o zadbaniu o to, aby obie grupy były reprezentatywne. W tym celu dzielimy użytkowników na 3 klastry za pomocą modelu KMeans. W grupowaniu skupiamy się na atrybutach związnych ze sposobem interakcji użytkownika z interfejsem. Następnie równomiernie wybieramy osobników z klastrów, tak aby utworzyć 2. grupy badawcze. Zakładamy, że takie podejście pozwoli nam uzyskać podobne grupy, zróżnicowanych użytkowników. 

Załadowanie danych dotyczących użytkowników


In [223]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

PATH_USERS = "data/v2/users.jsonl"
users_df = pd.read_json(PATH_USERS, lines=True) 

PATH_SESSION = "data/v2/sessions.jsonl"
session_df = pd.read_json(PATH_SESSION, lines=True)


Obliczenie długości trwania każdej sesji, na podstawie pliku session.jsonl


In [224]:
session_df['timestamp'] = pd.to_datetime(session_df['timestamp'])
session_df = session_df.sort_values(by='timestamp')
session_df['duration'] = session_df.groupby('session_id')['timestamp'].transform(lambda x: (x.max() - x.min()).total_seconds())
session_df

,timestamp,user_id,track_id,event_type,session_id,duration
17910,2022-11-24 08:27:56.000,145,5wDjsF1L2gLQ22kyvXiGUW,play,700,0.000
18562,2022-11-26 02:48:08.000,147,45S5WTQEGOB1VHr1Q4FuPl,play,727,2467.194
18563,2022-11-26 02:50:20.200,147,45S5WTQEGOB1VHr1Q4FuPl,skip,727,2467.194
18564,2022-11-26 02:50:20.200,147,,advertisement,727,2467.194
18565,2022-11-26 02:50:29.200,147,1EzrEOXmMH3G43AXT1y7pA,play,727,2467.194
...,...,...,...,...,...,...
4479,2023-11-24 11:31:30.883,112,0y60itmpH0aPKsFiGxmtnh,play,256,2948.816
4480,2023-11-24 11:34:43.532,112,0y60itmpH0aPKsFiGxmtnh,skip,256,2948.816
4481,2023-11-24 11:34:43.532,112,7fBv7CLKzipRk6EC6TWHOB,play,256,2948.816
4482,2023-11-24 11:34:58.099,112,7fBv7CLKzipRk6EC6TWHOB,like,256,2948.816


Zliczenie typów interakcji w ramach każdej sesji


In [225]:
session_summary_df = session_df.groupby('session_id').agg(
    duration=('duration', 'first'),
    user_id=('user_id', 'first'),
    num_play_events=('event_type', lambda x: (x == 'play').sum()),
    num_like_events=('event_type', lambda x: (x == 'like').sum()),
    num_skip_events=('event_type', lambda x: (x == 'skip').sum())
).reset_index()

session_summary_df = session_summary_df.set_index('session_id')
session_summary_df

,duration,user_id,num_play_events,num_like_events,num_skip_events
session_id,,,,,
124,5989.342,101,29,2,1
125,7315.030,101,36,4,3
126,6357.135,101,24,1,0
127,4042.551,101,20,0,3
128,1265.413,101,6,0,0
...,...,...,...,...,...
780,2212.385,150,15,0,8
781,3223.586,150,21,2,6
782,1493.654,150,9,0,1


Normalizacja wartości atrybutów i podział sesji na klastry

In [230]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = ['num_play_events', 'num_like_events', 'num_skip_events', 'duration']

num_clusters = 3
scaler = StandardScaler()
df_scaled = scaler.fit_transform(session_summary_df[features])

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
session_summary_df['cluster'] = kmeans.fit_predict(df_scaled)
session_summary_df

c:\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,duration,user_id,num_play_events,num_like_events,num_skip_events,cluster
session_id,,,,,,
124,5989.342,101,29,2,1,2
125,7315.030,101,36,4,3,2
126,6357.135,101,24,1,0,2
127,4042.551,101,20,0,3,2
128,1265.413,101,6,0,0,1
...,...,...,...,...,...,...
780,2212.385,150,15,0,8,1
781,3223.586,150,21,2,6,2
782,1493.654,150,9,0,1,1


Przydział użytkowników do klastrów


Dla każdego użytkownika sprawdzamy jakie klastry otrzymały powiązane z nim sesje i przydzielamy go do tej wsytępującej najczęściej


In [219]:
most_popular_cluster = session_summary.groupby('user_id')['cluster'].agg(lambda x: x.value_counts().idxmax())
users_df = users_df.merge(most_popular_cluster, how='left', on='user_id')
users_df

,user_id,name,city,street,favourite_genres,premium_user,cluster
0,101,Konrad Gajzler,Wrocław,pl. Żurawia 38,"[blues rock, brill building pop, argentine rock]",False,2
1,102,Szymon Ciereszko,Wrocław,pl. Kasprowicza 536,"[mpb, funk, new romantic]",False,2
2,103,Norbert Młodzik,Radom,ul. Lwowska 60/07,"[new wave, folk rock, country rock]",False,0
3,104,Patryk Boś,Warszawa,ul. Ciasna 53/93,"[tropical, folk rock, classic rock]",False,2
4,105,Elżbieta Strządała,Poznań,al. Niecała 10/94,"[post-teen pop, pop rock, latin rock]",False,1
5,106,Kajetan Jeszke,Kraków,pl. Jagodowa 862,"[classic rock, hard rock, quiet storm]",False,0
6,107,Mikołaj Dzikiewicz,Kraków,aleja Wróblewskiego 73/41,"[vocal jazz, adult standards, mpb]",False,2
7,108,Maciej Klinger,Warszawa,ul. Brzozowa 43/01,"[album rock, metal, quiet storm]",False,2
8,109,Ryszard Rzeszutko,Poznań,ulica Glówna 18,"[latin alternative, latin, ranchera]",True,0
9,110,Leon Pałyga,Kraków,ulica Malinowa 47/92,"[rock, latin pop, vocal jazz]",True,1


Podział użytkowników na grupy badawcze


In [220]:
sorted_users_df = users_df.copy()
sorted_users_df.sort_values('cluster', ascending=False, inplace=True)

group_A = pd.DataFrame(columns=sorted_users_df.columns)
group_B = pd.DataFrame(columns=sorted_users_df.columns)

temp = 0
for index, row in users_df.iterrows():
    if temp % 2 == 0:
        group_A.loc[index] = row.copy()
    if temp % 2 != 0:
        group_B.loc[index] = row.copy()
    temp += 1

Zapis grup do plików CSV

In [233]:
group_A.to_csv('./data/experiment/groupA.csv', index=False)
group_B.to_csv('./data/experiment/groupB.csv', index=False)

Obliczenie średniej długości sesji dla każdej grupy


In [231]:
import numpy as np 

def calculate_mean_duration_user(user):
    user_id = user['user_id']
    user_sessions_df = session_summary_df.loc[session_summary_df['user_id'] == user_id].copy()
    return np.mean(user_sessions_df['duration'])
    
def calculate_mean_duration_group(group):
    users_means = []
    for index, row in group.iterrows():
        mean_session_duration = calculate_mean_duration_user(row.copy())
        users_means.append(mean_session_duration)
    return np.mean(users_means)

print(f'Mean session duration for group A: {calculate_mean_duration_group(group_A)}')
print(f'Mean session duration for group B: {calculate_mean_duration_group(group_B)}')


Mean session duration for group A: 3408.306205630371
Mean seession duration fro group B: 3242.8546980233386


W biznesowym środowisku zakładamy nastepujący sposób przeprowadzania eksperymentu AB:

1. Na początku miesiąca przydzielamy użytkowników do grup badawczych za pomocą powyższego kodu;
2. Dla grupy B serwis Pozytywka zostaje wzbogacony o model klasyfikacyjny (bazowy lub docelowy);
3. Pod koniec miesiąca aktualizujemy dane o użytkownikach (w tym plik sessions.jsonl, niezbędny do przeprowadzenia eksperymentu);
4. W celu ewaluacji eksperymentu wyorzystujemy serwis, który zwórci nam informację o średnim czasie sesji każdej grupy.

Za pomocą wygenerowanych informacji, można jednoznacznie stwierdzić czy wprowadzony model z sukcesem doprowadził do wzrostu zaangażowania użytkownika (czasu spędzonego w serwisie).

Po wcześniejszym uruchomieniu serwisu z poziomu folderu services, za pomoca komendy: uvicorn main:app --reload, eksperyment mozna przeprowadzić wykonując poniższą komórkę


In [238]:
import requests
import json

response = requests.get("http://127.0.0.1:8000/ABexperiment")

if response.status_code == 200:
    response_data = json.loads(response.text)
    print(response_data)
else:
    print(f"Error: {response.status_code}")

{'no model': 3408.306205630371, 'model': 3242.8546980233386, 'time': '2024-01-19T22:40:50.817635'}
